In [1]:
import numpy as np
import csv
from copy import copy
import re

In [2]:
with open('Export_-_2018-01-17/Oligos_-_2018-01-17.csv', 'rb') as f:
    reader = csv.reader(f)
    sjoligos = list(reader)
    
with open('Export_-_2018-01-17/My_Oligos_-_2018-01-17.csv', 'rb') as f:
    reader = csv.reader(f)
    kltkoligos = list(reader)
    
with open('Export_-_2018-01-17/Elim_Primers_-_2018-01-17.csv', 'rb') as f:
    reader = csv.reader(f)
    elimoligos = list(reader)
    
with open('Export_-_2018-01-17/Sequetech_Primers_-_2018-01-17.csv', 'rb') as f:
    reader = csv.reader(f)
    seqoligos = list(reader)


In [3]:
alloligolist = sjoligos + kltkoligos + elimoligos + seqoligos
names = [item[0] for item in alloligolist]
sequences = [item[1] for item in alloligolist]
oligolist = zip(names, sequences)

In [4]:
import sys

In [5]:
#sys.setrecursionlimit(5000)

In [172]:
def complementary(seq):
    test_seq = seq.replace('G','c')
    test_seq = test_seq.replace('C','G')
    test_seq = test_seq.replace('c','C')
    test_seq = test_seq.replace('A','t')
    test_seq = test_seq.replace('T','A')
    test_seq = test_seq.replace('t','T')
    return test_seq

class Plasmid:
    def __init__(self, seq):
        self.sequence = seq
        self.comp = complementary(seq)
        self.blank_range = [0] * len(seq)
        self.oligo_list = {}
        self.complete_reads = []
        self.oligo_locs = {}
        self.num_iters = 0
        
    def range_int(self, st, en):
        self.target_seq = copy(self.sequence[st-1:en-1])
        self.target_range = [st-1, en] 
        
    def copy_pattern(self, oligos):
        #find each primers range
        #odd= start, even = end
        count = 1
        for i, o in enumerate(oligos):
            if self.blank_range[o[4][0]] == 0:
                self.blank_range[o[4][0]] = count
                self.oligo_list[count] = [o[0], o[3]]
                self.oligo_locs[count] = [o[4][0], o[4][1]]
                
                count += 1
                self.blank_range[o[4][1]] = count
                count += 1
            else:
                new_odd, new_even = self.adjuster(o[4][0], o[4][1])
                
                self.blank_range[new_odd] = count
                self.oligo_list[count] = [o[0], o[3]]
                self.oligo_locs[count] = [new_odd, new_even]
                
                count += 1
                self.blank_range[new_even] = count
                count += 1

    def adjuster(self, st, en):
        #if one of the original locations is occupied by another primer's range
        mag = 1
        vec = 1
        
        while True:
            new_st = st + (mag*vec)
            new_en = en + (mag*vec)
            
            if self.blank_range[new_st] == 0 and self.blank_range[new_en] == 0:
                print mag*vec
                return new_st, new_en
            elif vec > 0:
                vec *= -1
            else:
                vec *= -1
                mag += 1
    
    def find_last_odd(self, curr, lowlim):
        all_rem_odds = [x for x in self.blank_range[lowlim+1:curr] if x%2==1]
        
        if all_rem_odds == []:
            return ['','']
        
        last_odd = all_rem_odds[-1]
        new_curr = plasmid.blank_range.index(last_odd)
        return [last_odd, new_curr]
            
        
    
    def find_full_seq(self, curr_ind, low_ind, path):
        
        self.num_iters += 1
        
        if self.num_iters == 1000:
            return
        
        [next_odd, next_low] = self.find_last_odd(curr_ind, low_ind)

        print 'ITERATION ' + str(self.num_iters)
        print 'starting path ' + str(path)
        print 'next_odd ' + str(next_odd)
        print 'next_low ' + str(next_low)
        
        #if next_odd != '' and (next_low > self.target_range[1] or next_low < self.target_range[0] - 600):
        #if path == []:
            #print 'exit'
        #    return
        
        if next_odd != '':
            print 'odd found'
            path.append(next_odd)
            #print path
            curr_ind = self.blank_range.index(next_odd + 1)
            
            if curr_ind >= self.target_range[1]:
                print 'target reached'
                print path
                self.complete_reads.append(list(path))
                return
                curr_ind = self.oligo_locs[path[-1]][0]
                del path[-1]
                low_ind = self.oligo_locs[path[-1]][0]
                
                print 'next iter'
                print path
                print 'curr ind ' + str(curr_ind)
                print 'low ind ' + str(low_ind)
                self.find_full_seq(curr_ind, low_ind, path)
            
            else:
                
                print 'target not reached'
                print 'next iter'
                print 'curr ind ' + str(curr_ind)
                print 'low_ind ' + str(low_ind)
                self.find_full_seq(curr_ind, next_low, path)
                
        else:
            print 'no odd found'
            print path
            
            
            curr_ind = self.oligo_locs[path[-1]][0]
            del path[-1]
            
            print 'shortened path' + str(path)
            if path == []:
                return
            
            low_ind = self.oligo_locs[path[-1]][0]
            
            print 'next iter'
            print 'curr ind ' + str(curr_ind)
            print 'low_ind ' + str(low_ind)
            
            self.find_full_seq(curr_ind, low_ind, path)
    
    def rem_oligo(self, oli):
        self.blank_range[self.blank_range.index(oli)] = 0
        self.blank_range[self.blank_range.index(oli+1)] = 0
        
    def print_reads(self):
        for read in self.complete_reads:
            print '--------------'
            for item in read:
                print str(item) + ": " + str(self.oligo_list[item])
            print '--------------'
            
class Oligos:
    def __init__(self, oliglist):
        self.oligos = self.remove_size(oliglist)
        self.oligos = self.remove_name(oliglist)
        self.oligos = zip([item[0] for item in self.oligos], [item[1].upper() for item in self.oligos], [item[1].upper()[::-1] for item in self.oligos])
        
        self.binders = []
        self.find_bind()
        
    def remove_size(self, oliglist):
        return [x for x in oliglist if len(x[1]) < 35 and len(x[1]) > 15]
    
    def remove_name(self, oliglist):
        return [x for x in oligolist if len(x[0]) <= 12]
        
    def find_bind(self):
        for i, o in enumerate(self.oligos):
            bind = [m.end() for m in re.finditer(o[1], plasmid.sequence)]
            bind_rev = [m.start() for m in re.finditer(o[2], plasmid.comp)]
        
            if len(bind) + len(bind_rev) == 1:
                direction, ran = self.pattern(bind, bind_rev)
                self.binders.append((o[0], o[1], o[2], direction, ran))
        
    def pattern(self, b, br):
        if b == []:
            return '-', [br[0]-750, br[0]-50]
        elif br == []:
            return '+', [b[0]+50, b[0]+750]

In [173]:
with open('plasmid.txt', 'r') as f:
    plasmid_sequence = f.read()

plasmid = Plasmid(plasmid_sequence.upper())
range_start = 1050
range_end = 5114
plasmid.range_int(range_start,range_end)

In [174]:
plasmid.oligo_locs[49][0]

KeyError: 49

In [175]:
o = Oligos(oligolist)
print o.binders[4][0]

oSJ74


In [176]:
for og in o.oligos:
    print len(og[0])

4
5
5
5
5
5
5
6
4
4
4
4
5
5
5
4
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
12
4
4
10
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
8
4
4
4
4
4
4
4
4
4
6
4
4
4
4
4
4
7
4
6
4
4
4
4
4
6
4
4
4
4
4
4
12
4
4
4
4
4
4
4
6
4
4
6
6
6
7
7
7
7
4
4
12
4
4
6
4
6
6
6
6
7
9
9
4
10
4
4
6
4
4
4
4
4
4
4
4
4
4
6
6
6
7
4
4
4
4
8
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6
3
12
11
7
7
4
4
7
7
7
12
7
7
4
7
4
10
6
6
6
6
6
6
6
6
6
6
6
7
4
4
4
6
6
6
6
6
6
6
6
6
7
6
4
10
7
7
7
7
6
6
6
6
11
4
4
4
4
4
4
6
7
4
4
6
6
5
5
4
4
4
4
4
4
4
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
4
4
4
4
4
4
4

In [177]:
plasmid.copy_pattern(o.binders)
print plasmid.blank_range
print plasmid.oligo_list

1
1
1
1
1
1
1
1
-1
1
1
1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 144, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 145, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [178]:
[906, 1713]
print plasmid.blank_range.index(133)
print plasmid.blank_range.index(87)
print plasmid.blank_range.index(25)
print plasmid.blank_range.index(26)
print plasmid.blank_range.index(73)

print [x for x in plasmid.blank_range[901:1204] if x%2==1]

5901
374
901
1601
5069
[25, 65, 45, 113, 79, 89, 119, 129, 95, 139, 21, 83]


In [186]:

plasmid.find_full_seq(1070, 1, [])

ITERATION 17
starting path []
next_odd 95
next_low 1013
odd found
target not reached
next iter
curr ind 1713
low_ind 1
ITERATION 18
starting path [95]
next_odd 47
next_low 1712
odd found
target not reached
next iter
curr ind 2412
low_ind 1013
ITERATION 19
starting path [95, 47]
next_odd 37
next_low 2397
odd found
target not reached
next iter
curr ind 3097
low_ind 1712
ITERATION 20
starting path [95, 47, 37]
next_odd 69
next_low 2462
odd found
target not reached
next iter
curr ind 3162
low_ind 2397
ITERATION 21
starting path [95, 47, 37, 69]
next_odd 49
next_low 3161
odd found
target not reached
next iter
curr ind 3861
low_ind 2462
ITERATION 22
starting path [95, 47, 37, 69, 49]
next_odd 33
next_low 3319
odd found
target not reached
next iter
curr ind 4019
low_ind 3161
ITERATION 23
starting path [95, 47, 37, 69, 49, 33]
next_odd 51
next_low 3998
odd found
target not reached
next iter
curr ind 4698
low_ind 3319
ITERATION 24
starting path [95, 47, 37, 69, 49, 33, 51]
next_odd 135
next_low

In [187]:
plasmid.print_reads()

--------------
139: ['attL1.AP', '+']
107: ['t486', '+']
37: ['oSJ106', '+']
57: ['oSJ215', '+']
49: ['oSJ183', '+']
33: ['oSJ104', '+']
51: ['oSJ184', '+']
135: ['3pLG112', '-']
--------------
--------------
95: ['t290', '+']
47: ['oSJ148', '+']
37: ['oSJ106', '+']
57: ['oSJ215', '+']
49: ['oSJ183', '+']
33: ['oSJ104', '+']
51: ['oSJ184', '+']
135: ['3pLG112', '-']
--------------
--------------
95: ['t290', '+']
47: ['oSJ148', '+']
37: ['oSJ106', '+']
69: ['oSJ230', '+']
49: ['oSJ183', '+']
33: ['oSJ104', '+']
51: ['oSJ184', '+']
135: ['3pLG112', '-']
--------------


In [83]:
print len(o.binders)

for i,oli in enumerate(o.binders):
    print oli

72
('oSJ11', 'CGCATCGAGCTGAAGGGCAT', 'TACGGGAAGTCGAGCTACGC', '+', [4737, 5437])
('oSJ12', 'TTAGGCGGCGGTCACGAACT', 'TCAAGCACTGGCGGCGGATT', '-', [4218, 4918])
('oSJ1', 'GCTGGAGTTCGTGACCGCCGCCTAA', 'AATCCGCCGCCAGTGCTTGAGGTCG', '+', [5038, 5738])
('oSJ2', 'AGTCGATGCCCTTCAGCTCGATGCG', 'GCGTAGCTCGACTTCCCGTAGCTGA', '-', [3917, 4617])
('oSJ7', 'CAGGATGTTGCCGTCCTCCTTGAAGTCGATG', 'GTAGCTGAAGTTCCTCCTGCCGTTGTAGGAC', '-', [3934, 4634])
('oSJ74', 'TTGCTCAGGGCGGACTGATGGCTCA', 'ACTCGGTAGTCAGGCGGGACTCGTT', '-', [4155, 4855])
('oSJ78', 'TTGCCGTCCTCCTTGAAGTC', 'CTGAAGTTCCTCCTGCCGTT', '-', [3938, 4638])
('oSJ29', 'TAAGCGGCCGCACTCGAGAT', 'TAGAGCTCACGCCGGCGAAT', '+', [5061, 5761])
('oSJ8', 'CAGTCCGCCCTGAGCAAAGACCCCAA', 'AACCCCAGAAACGAGTCCCGCCTGAC', '+', [4989, 5689])
('oSJ36', 'GGTGGAGCCTGCTTTTTTGT', 'TGTTTTTTCGTCCGAGGTGG', '-', [384, 1084])
('oSJ40', 'CTTTGTACAAAAAAGCAGGC', 'CGGACGAAAAAACATGTTTC', '+', [1198, 1898])
('oSJ55', 'GGTGGAGCCTGCTTTTTTGTACAAA', 'AAACATGTTTTTTCGTCCGAGGTGG', '-', [379, 1079])
('oSJ

In [84]:
for oligo in oligolist:
    [m.start() for m in re.finditer(, plasmid_sequence)]

SyntaxError: invalid syntax (<ipython-input-84-bceafa6221a6>, line 2)

In [ ]:
oSJ5.sequence

In [ ]:
testuniv = range(0,10)
nouniv = np.arange(101,200)
subs = []
for i in testuniv:
    subs.append(range(i,i+3))
    

In [ ]:
print subs

In [ ]:
new = []
new = [s for s in subs if s != subs[0]]
print new

In [ ]:
def recur(univ, subsets, templist):
    remuniv = copy(univ)
    for s in subsets:
        overlap = list(set(s) & set(remuniv))
        
        if overlap != []:
            remuniv = [x for x in univ if x not in overlap]
            remsets = [l for l in subsets if l != s]
            templist.append(s)
            
            if remuniv == []:
                print "one"
                answer.append(templist)
                return templist
            else:
                new = recur(remuniv, remsets, templist)
                del templist[-1]
                

In [ ]:
answer = []
tempin = []
recur(testuniv, subs, tempin)

In [ ]:
print answer[0]

In [ ]:
j=0
for i, o in enumerate(oligolist):
    bind = [m.start() for m in re.finditer(o[1].upper(), plasmid.sequence)]
    bind_rev = [m.start() for m in re.finditer(o[1].upper()[::-1], plasmid.comp)]

    if len(bind) + len(bind_rev) > 0:
        j +=1
        print i
        print o[0], o[1], bind, bind_rev
        print zip(o[0], o[1], bind, bind_rev)
        
print j